### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__UNESCO Syndication__  
  URL:  
  ・https://whc.unesco.org/en/syndication    
  DATA:  
  ・whc-sites-2021.xls  
  データの取得方法:  
    「The List」にある「XLS」リンクをクリックします。

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import requests
import seaborn as sns

`plotly`のグラフが表示されない場合、デフォルトのrendererの設定を変更します。

In [ ]:
pio.renderers

たとえばJupyter Notebookで`plotly`のグラフが表示されない場合は`'notebook'`を設定します。

参考）  
Jupyter Classic Notebook Problems  
https://plotly.com/python/troubleshooting/#jupyter-notebook-classic-problems

In [ ]:
# pio.renderers.default='notebook'

ノートブックのレイアウト定義。

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;} # markdownの表を左寄せにする。
</style>

### 1. データの概要を掴む

世界遺産データをロード。

In [ ]:
world_heritage_df = pd.read_excel('data/whc-sites-2021.xls')

len(world_heritage_df)

データの中身を確認。

In [ ]:
world_heritage_df['danger_list']

In [ ]:
world_heritage_df.head()

各列の状況を`info()`で確認。

各列が何を意味しているか分かりますか？

参考）  
・Template:UNESCO World Heritage Site row/doc  
　https://en.wikipedia.org/wiki/Template:UNESCO_World_Heritage_Site_row/doc

・UNESCO The Criteria for Selection  
　https://whc.unesco.org/en/criteria/

・日本ユネスコ協会連盟 登録の基準  
　https://www.unesco.or.jp/activities/isan/decides/

In [ ]:
world_heritage_df.info()

また、どんな世界遺産があるのかUNESCOのサイトで確認してみましょう。  
`states_name_en`列にある任意の国名で絞って`id_no`を取得し、ギャラリーへのリンクのリストを作成してみます。  

まず`states_name_en`にどんな国名があるか`unique()`で確認します。

In [ ]:
world_heritage_df['states_name_en'].unique()

複数の国に跨っているケースがあって、その場合はカンマ区切りになっているようなので、それぞれを独立した行にしてしまいます。

手順は以下です。  
1.`states_name_en`列の値を行ごとにカンマで分割してリストにし、新たな列`single_country`に格納。  
2.`single_country`列に`explode()`を実行し、リストの要素ごとの行を作成する。

まず列`single_country`にリストを格納します。

In [ ]:
world_heritage_df = world_heritage_df \
                     .assign(single_country = world_heritage_df['states_name_en'].str.split(','))

world_heritage_df['single_country']

次に`explode()`で`single_country`列のリストから複数の行を生成します。

`explode()`の実行前に現在のデータ件数を確認しておきます。

In [ ]:
print(f"explode実行前: {len(world_heritage_df):,}件")

`explode()`を実行します。

In [ ]:
world_heritage_df = world_heritage_df.explode('single_country')

print(f"explode実行後: {len(world_heritage_df):,}件")

world_heritage_df['single_country']

`single_country`列の値を`unique()`で確認します。

In [ ]:
world_heritage_df['single_country'].unique()

それではギャラリーURLのリストを作成します。  
URLのパターンは以下のようになります。

`https://whc.unesco.org/en/list/<id_no>/gallery/`

In [ ]:
# 任意の国を指定
target_country = 'Zambia'

heritage_selected_df = world_heritage_df[world_heritage_df['single_country'] == target_country]
for i, row in heritage_selected_df.iterrows():
    print(f"・{row['name_en']}")
    print(f"　https://whc.unesco.org/en/list/{row['id_no']}/gallery/")

### 2. groupbyによる集約

ここでは、`groupby()`を利用してデータを集約してみます。

#### 2-1. 簡単な集計関数の利用

国別の世界遺産数を取得し、多い順にソートします。

`groupby()`で`DataFrameGroupBy`オブジェクトを生成し、`size()`で国ごとの件数を計算します。

In [ ]:
groupby_country_size = world_heritage_df.groupby('single_country', as_index=False) \
                        .size() \
                        .sort_values('size', ascending=False)

groupby_country_size.head(20)

横棒グラフですべての国の世界遺産数を多い順に表示してみましょう。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(20,50))
sns.barplot(x="size", y="single_country", data=groupby_country_size)

同様に地域でグループ化し、多い順に横棒グラフで表示してみます。

複数の国に跨っているものを複数の行にさせたので`id_no`の重複がありますから、`drop_duplicates()`を使って`id_no`の重複のないデータフレームを別途用意します。

In [ ]:
world_heritage_unique_id_no_df = world_heritage_df.drop_duplicates(subset=['id_no'])

print(f"world_heritage_df: {len(world_heritage_df):,}件")
print(f"world_heritage_unique_id_no_df: {len(world_heritage_unique_id_no_df):,}件")

地域でグループ化して遺産数を計算し、多い順に並べます。

In [ ]:
groupby_region_size = world_heritage_unique_id_no_df.groupby('region_en')['id_no'].size().reset_index()

groupby_region_size = groupby_region_size.rename(columns={'id_no': 'size'}) \
                         .sort_values('size', ascending=False)

groupby_region_size

グラフを表示します。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))

sns.barplot(x="size", y="region_en", data=groupby_region_size)

#### 2-2. agg関数の利用

列ごとに複数の任意の集約関数を利用したい場合、`agg()`を辞書を引数にして呼び出します。

世界遺産の登録日の最大値、最小値、平均とヘクタール面積の最大値、最小値、中央値を`agg()`を使って取得してみましょう。

In [ ]:
world_heritage_df.agg({
    'date_inscribed': ['max', 'min', 'mean'],
    'area_hectares': ['max', 'min', 'median']
}).round(2)

もちろんこの場合は`agg()`を使わずに`describe()`で簡単に計算できます。

In [ ]:
world_heritage_df[['date_inscribed','area_hectares']].describe().round(2)

#### 2-3. apply関数の利用

任意の関数を集約単位で実行したい場合、`apply()`を利用します。

地域ごと、国ごとの危機遺産率を取得してみましょう。

最初に、引数として遺産データフレームを取って危機遺産率を計算する関数を定義します。

In [ ]:
def get_danger_rate(heritage_df):
    """危機遺産率を取得。

    Args:
        heritage_df (pandas.DataFrame): 遺産データフレーム。
    Returns:
        numpy.float64: 危機遺産率。
    """

    return round((heritage_df['danger'] == 1).sum() / len(heritage_df), 2)

関数をデータ全体に対して実行してみましょう。

In [ ]:
get_danger_rate(world_heritage_df)

`groupy()`で`DataFrameGroupBy`オブジェクトを取得し、作成した関数を引数として`apply()`を呼び出すと、集約単位ごとに危機遺産率を計算することができます。

地域ごとの危機遺産率を取得し、降順で表示してみましょう。  
地域ごとですので`id_no`の重複のないデータフレームを利用します。

どんなことが読み取れますか？

In [ ]:
### チャレンジしてみましょう！！ ###

同様に国ごとの危機遺産率を取得し、降順で表示してみましょう。

In [ ]:
groupby_country_danger = world_heritage_df.groupby('single_country').apply(get_danger_rate) \
                          .sort_values(ascending=False)

groupby_country_danger.head(40)

国ごとの危機遺産率を以下のルールで地図上に表示します。

- `plotly`を使って地球儀を表示。
- 危機遺産率が0でないものが対象。
- 国の位置を赤い点で示し、危険遺産率に合わせて色を変化させる。
- 国の位置座標は大雑把に国ごとの世界遺産の緯度、経度の平均値とする。

まず世界遺産のデータフレームを国でグループ化し、緯度、経度の平均を計算したデータフレームを作成します。

ここでは`agg()`にキーワード引数を渡して呼び出してみます。

In [ ]:
country_danger_rate_df = world_heritage_df.groupby('single_country').agg(
                          lat=('latitude', 'mean'),
                          lon=('longitude', 'mean'))

country_danger_rate_df

作成したデータフレームに、先ほどの危機遺産率を新たな列`danger_rate`として加えます。

In [ ]:
country_danger_rate_df['danger_rate'] = groupby_country_danger

country_danger_rate_df

危機遺産率が0のものを取り除き、インデックスをリセットします。

In [ ]:
# 危機率が0より大きい
country_danger_rate_non_zero_df = country_danger_rate_df[country_danger_rate_df['danger_rate'] > 0]

# インデックスをリセット
country_danger_rate_non_zero_df = country_danger_rate_non_zero_df.reset_index()

country_danger_rate_non_zero_df.head()

それでは`plotly`の`px.scatter_geo()`に引数`projection="orthographic"`を渡して地球儀を表示します。

`plotly`のグラフが表示されない場合、デフォルトのrendererの設定を変更します。


どんなことが読み取れますか？

In [ ]:
fig = px.scatter_geo(country_danger_rate_non_zero_df,
                     lat='lat',
                     lon='lon',
                     color='danger_rate',
                     hover_data=['single_country'],
                     projection="orthographic",
                     color_continuous_scale = 'Pinkyl',
                     range_color=[0, 1])

center = {'lat':0, 'lon':139.7671} # 経度は東京駅
fig.update_geos(scope='world',
                center=center,
                lonaxis_range=[center['lon']-1, center['lon']+1])

fig.show()

### 3. 横持ち・縦持ちへの変換

ここでは、データの各行に１つずつの情報を持つ縦持ちの状態からクロス集計表のような横持ちの状態への変換、またはその逆への変換をおこないます。

#### 3-1. 横持ちへの変換

縦軸を国、横軸をカテゴリとするクロス集計表を作ってみましょう。  
クロス集計とは、アンケート結果などを集計する際に2つ以上の観点でまとめる統計手法のことです。

まずは国、カテゴリでグループ化して件数を出力します。

In [ ]:
groupby_country_category = world_heritage_df.groupby(["single_country", "category"]).size()

groupby_country_category

横持ちになるよう、`unstack()`でカテゴリのそれぞれの値を列にします。  
また、該当する値がない場合は0とします。

これでクロス集計は完成です。

In [ ]:
### チャレンジしてみましょう！！ ###

`groupby()`を使わず、`pd.crosstab()`を使う方法でもやってみましょう。

In [ ]:
pd.crosstab(world_heritage_df["single_country"], world_heritage_df["category"])

さらに`pd.pivot_table()`でも同じクロス集計を作成してみましょう。

数値計算でどんな関数を使うのか指定する必要があります。

In [ ]:
country_category_pivot_table = pd.pivot_table(world_heritage_df,
                               index=["single_country"],
                               columns="category",
                               values="id_no",
                               aggfunc=lambda x: len(x),
                               fill_value=0)

country_category_pivot_table

最後に積み上げ横棒グラフでデータを多い順に表示してみましょう。

まず国ごとにカテゴリ数を合計して新たな列`Total`に格納後、`Total`列の値の降順に並べ替えます。

In [ ]:
country_category_pivot_table['Total'] = country_category_pivot_table.sum(axis=1)

country_category_pivot_table = country_category_pivot_table.sort_values('Total', ascending=False)

country_category_pivot_table

グラフを表示します。

どんなことが読み取れますか？

In [ ]:
palette = sns.color_palette('Set2', len(country_category_pivot_table.columns))

# 配列に数値を積み上げていく。
left = np.zeros(len(country_category_pivot_table.index))

fig, ax = plt.subplots(figsize=(20,50))

for i, column in enumerate(country_category_pivot_table.columns[0:3]):
    sns.barplot(x=country_category_pivot_table[column],
                y=country_category_pivot_table.index,
                color=palette[i],
                label=column,
                left=left,
                ax=ax)

    left += list(country_category_pivot_table[column])

ax.set_xlabel("count")
ax.legend(loc="upper left", bbox_to_anchor=(1,1))

積み上げ横棒グラフの作成は`pandas.DataFrame.plot()`を利用した方が簡単です。

In [ ]:
country_category_pivot_table \
    .sort_values('Total', ascending=True) \
    .iloc[:,0:3] \
    .plot(kind='barh', stacked=True, figsize=(20, 50))

#### 3-2. 縦持ちへの変換

今度は手順がやや複雑になりますが、地域別のC1〜C6、N7〜N10の登録基準の合計数をレーダーチャートで表示してみます。

手順は以下になります。  
1.`id_no`の重複のないデータフレームから縦軸を地域、横軸を登録基準とする横持ちのデータフレームを作成。  
2.地域のうち、複数の地域に跨るものを単一の地域の加算した後除去。  
3.データフレームを整形し、最終的に「地域」「登録基準」「合計数」の列を持つ縦持ちにする。  
4.`ploty`の`line_polar()`でレーダーチャートを表示する。  

まず横持ちのデータフレームを作成します。

In [ ]:
groupby_region_criteria_sum = world_heritage_unique_id_no_df \
                              .filter(regex='^(region_en|(C|N)\d+)$', axis=1) \
                              .groupby('region_en') \
                              .sum()

groupby_region_criteria_sum

次に複数の地域に跨るものの数値をそれぞれの単一の地域に加算します。

In [ ]:
groupby_region_criteria_sum.loc['Asia and the Pacific'] = \
    groupby_region_criteria_sum.loc['Asia and the Pacific'] \
    + groupby_region_criteria_sum.loc['Asia and the Pacific,Europe and North America']

groupby_region_criteria_sum.loc['Europe and North America'] = \
    groupby_region_criteria_sum.loc['Europe and North America'] \
    + groupby_region_criteria_sum.loc['Asia and the Pacific,Europe and North America']

groupby_region_criteria_sum.loc['Asia and the Pacific'] = \
    groupby_region_criteria_sum.loc['Asia and the Pacific'] \
    + groupby_region_criteria_sum \
        .loc['Asia and the Pacific,Europe and North America,Latin America and the Caribbean']

groupby_region_criteria_sum.loc['Europe and North America'] = \
    groupby_region_criteria_sum.loc['Europe and North America'] \
    + groupby_region_criteria_sum \
        .loc['Asia and the Pacific,Europe and North America,Latin America and the Caribbean']

groupby_region_criteria_sum.loc['Latin America and the Caribbean'] = \
    groupby_region_criteria_sum.loc['Latin America and the Caribbean'] \
    + groupby_region_criteria_sum \
        .loc['Asia and the Pacific,Europe and North America,Latin America and the Caribbean']

インデックスから複数の地域に跨るものを除去します。

In [ ]:
groupby_region_criteria_sum_multi_region_removed = groupby_region_criteria_sum.drop(index=[
                                'Asia and the Pacific,Europe and North America',
                                'Asia and the Pacific,Europe and North America,Latin America and the Caribbean'])

groupby_region_criteria_sum_multi_region_removed

登録基準をインデックスにした後、すべてのインデックスをリセットして縦持ちへの変換を完了します。

In [ ]:
region_criteria_sum_df = groupby_region_criteria_sum_multi_region_removed.stack().reset_index()

region_criteria_sum_df.columns = ['region', 'criteria', 'sum']

region_criteria_sum_df.head(10)

`plotly`の`line_ploar()`レーダーチャートを表示します。

登録基準の説明を参考にしつつ、どんなことが読み取れますか？

|criteria|英語（[UNESCO The Criteria for Selection](https://whc.unesco.org/en/criteria/)より転載）|日本語（[日本ユネスコ協会連盟 登録の基準](https://www.unesco.or.jp/activities/isan/decides/)より転載）|
|:--:|:--|:--|
|C1|to represent a masterpiece of human creative genius;|人間の創造的才能を表す傑作である。|
|C2|to exhibit an important interchange of human values, over a span of time or within a cultural area of the world, on developments in architecture or technology, monumental arts, town-planning or landscape design;|建築、科学技術、記念碑、都市計画、景観設計の発展に重要な影響を与えた、ある期間にわたる価値観の交流又はある文化圏内での価値観の交流を示すものである。|
|C3|to bear a unique or at least exceptional testimony to a cultural tradition or to a civilization which is living or which has disappeared;|現存するか消滅しているかにかかわらず、ある文化的伝統又は文明の存在を伝承する物証として無二の存在(少なくとも希有な存在)である。|
|C4|to be an outstanding example of a type of building, architectural or technological ensemble or landscape which illustrates (a) significant stage(s) in human history;|歴史上の重要な段階を物語る建築物、その集合体、科学技術の集合体、あるいは景観を代表する顕著な見本である。|
|C5|to be an outstanding example of a traditional human settlement, land-use, or sea-use which is representative of a culture (or cultures), or human interaction with the environment especially when it has become vulnerable under the impact of irreversible change;|あるひとつの文化(または複数の文化)を特徴づけるような伝統的居住形態若しくは陸上・海上の土地利用形態を代表する顕著な見本である。又は、人類と環境とのふれあいを代表する顕著な見本である(特に不可逆的な変化によりその存続が危ぶまれているもの)|
|C6|to be directly or tangibly associated with events or living traditions, with ideas, or with beliefs, with artistic and literary works of outstanding universal significance. (The Committee considers that this criterion should preferably be used in conjunction with other criteria);|顕著な普遍的価値を有する出来事(行事)、生きた伝統、思想、信仰、芸術的作品、あるいは文学的作品と直接または実質的関連がある(この基準は他の基準とあわせて用いられることが望ましい)。|
|N7|to contain superlative natural phenomena or areas of exceptional natural beauty and aesthetic importance;|最上級の自然現象、又は、類まれな自然美・美的価値を有する地域を包含する。|
|N8|to be outstanding examples representing major stages of earth's history, including the record of life, significant on-going geological processes in the development of landforms, or significant geomorphic or physiographic features;|生命進化の記録や、地形形成における重要な進行中の地質学的過程、あるいは重要な地形学的又は自然地理学的特徴といった、地球の歴史の主要な段階を代表する顕著な見本である。|
|N9|to be outstanding examples representing significant on-going ecological and biological processes in the evolution and development of terrestrial, fresh water, coastal and marine ecosystems and communities of plants and animals;|陸上・淡水域・沿岸・海洋の生態系や動植物群集の進化、発展において、重要な進行中の生態学的過程又は生物学的過程を代表する顕著な見本である。|
|N10|to contain the most important and significant natural habitats for in-situ conservation of biological diversity, including those containing threatened species of outstanding universal value from the point of view of science or conservation.|学術上又は保全上顕著な普遍的価値を有する絶滅のおそれのある種の生息地など、生物多様性の生息域内保全にとって最も重要な自然の生息地を包含する。|

In [ ]:
%%capture --no-display

fig = px.line_polar(region_criteria_sum_df, r="sum", theta="criteria", color='region', line_close=True)

fig.show()

同様に任意の国を幾つか選んで、１つのレーダーチャート上に表示してみましょう。

まずは「国」「登録基準」「合計数」の列を持つ縦持ちのデータフレームを作成します。

In [ ]:
### チャレンジしてみましょう！！ ###

作成したデータフレームを使ってレーダーチャートを表示します。

In [ ]:
### チャレンジしてみましょう！！ ###

### 4. 世界遺産登録の変遷を地図上に表示

`plotly`のアニメーション機能を使って、世界遺産登録の変遷を地図上に表示してみましょう。

年が進むにつれて各国の総登録数が増えていき、国ごとの区画の色が濃くなるようにします。

まず最初に国名コードのデータを取得します。

地図上の国の区画とデータは3文字の国名コード（ISO 3166-1 alpha-3）で結びつけるのですが、世界遺産データの2文字の国名コードしかありませんので、データをウィキペディアから取得しロードします。

一度取得したデータは、何度も取得しないように`data/country_code.csv`保存するようにしましょう。


ウィキペディア ISO 3166-1  
https://ja.wikipedia.org/wiki/ISO_3166-1


In [ ]:
country_code_wikipedia_url = 'https://ja.wikipedia.org/wiki/ISO_3166-1'
country_code_csv_path = 'data/country_code.csv'

try:
    country_code_df = pd.read_csv(country_code_csv_path)
    print(f"ファイル'{country_code_csv_path}'からデータを取得しました。")
except FileNotFoundError:
    print(f"ファイル'{country_code_csv_path}'が存在しないためウィキペディアからデータを取得します。")
    html = requests.get(country_code_wikipedia_url)
    country_code_df = pd.read_html(html.text)[0]
    country_code_df.to_csv(country_code_csv_path, index=False)

len(country_code_df)

取得したデータの中身を表示。

In [ ]:
country_code_df.head()

１行目を削除し、インデックスをリセットします。

それから英語の国名を`country`、3文字のコードを`country_code`として、他の必要ない列は削除します。

In [ ]:
# 1行目を削除
country_code_df = country_code_df.iloc[1:]

# インデックスをリセット
country_code_df.reset_index(drop=True, inplace=True)

# 必要のない列を削除
country_code_df = country_code_df[['ISO 3166-1における英語名', 'alpha-3']]

# 列名を変更
country_code_df.columns = ['country', 'country_code']

country_code_df.head()

次に地図用のデータフレームを用意します。  
以下の4つの属性を縦持ちしたデータフレームが必要です。

|#|属性|説明|
|:---:|:---|:---|
|1|国名コード|世界遺産の登録があるすべての国の国名コード。|
|2|国名|世界遺産データにある英語名。|
|3|年度|世界全体で一番最初の世界遺産の登録年の前年から最新の登録の年度まで。すべての国が、すべての年度の行を持つ必要がある。|
|4|登録数累計|各国の各年度ごとの登録数累計。|

作成手順は以下のようになります。

1. 世界遺産データを国、登録年でグループ化し、件数を集計した`Series`オブジェクトを作成。
2. 世界遺産の登録があるすべての国とすべての年度を掛け合わせたインデックスを持つ、値が空の`Series`オブジェクトを作成。
3. 2を1で上書きし、値が空の部分はすべて0で埋める。
4. 国、年度ごとに累計を計算し、新たな列として加える。
5. 国、年度のインデックスをリセットして列に変換し、データフレームにする。
6. 国名をキーに国名コードのデータフレームと結合する。

世界遺産データを国、登録年でグループ化し、件数を集計した`Series`オブジェクトを作成します。

In [ ]:
country_year_inscribed_s = world_heritage_df.groupby(['single_country', 'date_inscribed']).size()

country_year_inscribed_s

世界遺産の登録があるすべての国とすべての年度を掛け合わせたインデックスを持つ、値が空の`Series`オブジェクトを作成します。

年度は世界全体で一番最初の世界遺産の登録年の前年から最新の登録の年度までです。

In [ ]:
countries = world_heritage_df['single_country'].unique()
year_range = range(world_heritage_df['date_inscribed'].min()-1, world_heritage_df['date_inscribed'].max()+1)

country_year_index = pd.MultiIndex.from_product([countries, year_range], \
                                                names=['single_country', 'date_inscribed'])
country_year_s = pd.Series(index=country_year_index, dtype='float64')

country_year_s

作成した空の`Series`オブジェクトを先ほどの件数のある`Series`オブジェクトで上書きし、値が空の部分はすべて0で埋めます。

上書きには`combine_first()`を使います。

In [ ]:
country_year_combined_s = country_year_s.combine_first(country_year_inscribed_s).fillna(0)

country_year_combined_s

国、年度ごとに累計を計算し、新たな列として加えます。

In [ ]:
country_year_cumsum_s = country_year_combined_s.groupby(level='single_country').cumsum()

country_year_cumsum_s

`Zimbabwe`（ジンバブエ）のデータで正しく累計が計算されているか確認します。

In [ ]:
country_year_cumsum_s['Zimbabwe']

問題なければ国、年度のインデックスをリセットして列に変換し、データフレームにします。

各列名は`country`、`year`、`total`とします。

In [ ]:
country_year_total_df = country_year_cumsum_s.reset_index()
country_year_total_df.columns = ['country', 'year', 'total']

country_year_total_df.head()

今度は国名をキーに国名コードのデータフレームと結合します。

In [ ]:
country_year_total_code_df = pd.merge(country_year_total_df, country_code_df, how='left', on="country")

country_year_total_code_df.head()

国名コードがない行があるか確認し、あればその国名を取得します。

In [ ]:
country_year_total_code_df[country_year_total_code_df['country_code'].isna()]['country'].unique()

コードが見つからなかった国名と、正しいコードは以下になります。  
１つ１つ登録してしまいます。

- "Democratic People's Republic of Korea": PRK
- 'Democratic Republic of the Congo': COD
- 'Gambia (the)': GMB
- 'Jerusalem (Site proposed by Jordan)': JOR
- 'Palestine': PSE
- 'Republic of Korea': KOR
- 'United Republic of Tanzania': TZA
- 'Republic of Moldova': MDA

In [ ]:
country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == "Democratic People's Republic of Korea", 'country_code'] = 'PRK'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Democratic Republic of the Congo', 'country_code'] = 'COD'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Gambia (the)', 'country_code'] = 'GMB'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Jerusalem (Site proposed by Jordan)', 'country_code'] = 'JOR'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Palestine', 'country_code'] = 'PSE'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Republic of Korea', 'country_code'] = 'KOR'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'United Republic of Tanzania', 'country_code'] = 'TZA'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Republic of Moldova', 'country_code'] = 'MDA'

country_year_total_code_df.loc[ \
    country_year_total_code_df['country'] == 'Türkiye', 'country_code'] = 'TUR'

もう一度、コードがない国がないか確認します。

In [ ]:
country_year_total_code_df['country_code'].isna().sum()

それでは`plotly`の`px.choropleth()`を利用して地図を表示します。

どんなことが読み取れますか？

In [ ]:
fig = px.choropleth(country_year_total_code_df,
                    locations="country_code",
                    color="total",
                    hover_name="country",
                    animation_frame="year",
                    projection="natural earth",
                    color_continuous_scale = 'Oranges',
                    range_color=[1,40])   

center = {'lat':0, 'lon':139.7671} # 緯度は東京駅
fig.update_geos(scope='world',
                center=center,
                lonaxis_range=[center['lon']-180, center['lon']+180])

fig.show()